# Densepose extract

In [58]:
%%time
import os
from os.path import exists, join, basename, splitext

# install Anaconda Python 2.7 to control the dependencies
# see for more info: 
if not exists('anaconda2'):
  !wget -q https://repo.anaconda.com/archive/Anaconda2-2019.03-Linux-x86_64.sh
  !chmod +x Anaconda2-2019.03-Linux-x86_64.sh
  !bash ./Anaconda2-2019.03-Linux-x86_64.sh -b -f -p /content/anaconda2
  # set PATH environment variable
  os.environ['PATH'] = "/content/anaconda2/bin:" + os.environ['PATH']
  # install PyTorch
  !conda install -y pyyaml=3.12
  !conda install -y mkl-include
  !conda install -y pytorch=1.0.1 torchvision cudatoolkit=10.0 -c pytorch
  !ln -s /content/anaconda2/lib/python2.7/site-packages/torch/lib/ /content/anaconda2/lib/python2.7/site-packages/
  # install GCC 4.9
  !conda install -y -c serge-sans-paille gcc_49
  !ln -fs /content/anaconda2/lib/libmpfr.so /content/anaconda2/lib/libmpfr.so.4
  os.environ['CC'] = '/content/anaconda2/bin/gcc-4.9'
  os.environ['CXX'] = '/content/anaconda2/bin/g++-4.9'
  # protobuf 3.5
  #!apt-get -qq remove -y protobuf-compiler
  !conda install -y protobuf=3.5
  # pycocotools
  !conda install -y -c conda-forge pycocotools
  
# we need some headers from the pytorch source
if not exists('pytorch'):
  !git clone -q --depth 1 --recursive -b v1.0.1 https://github.com/pytorch/pytorch

PREFIX=/content/anaconda2
installing: python-2.7.16-h9bab390_0 ...
using -f (force) option
Python 2.7.16 :: Anaconda, Inc.
installing: conda-env-2.6.0-1 ...
using -f (force) option
installing: blas-1.0-mkl ...
using -f (force) option
installing: ca-certificates-2019.1.23-0 ...
using -f (force) option
installing: intel-openmp-2019.3-199 ...
using -f (force) option
installing: libgcc-ng-8.2.0-hdf63c60_1 ...
using -f (force) option
installing: libgfortran-ng-7.3.0-hdf63c60_0 ...
using -f (force) option
installing: libstdcxx-ng-8.2.0-hdf63c60_1 ...
using -f (force) option
installing: bzip2-1.0.6-h14c3975_5 ...
using -f (force) option
installing: expat-2.2.6-he6710b0_0 ...
using -f (force) option
installing: fribidi-1.0.5-h7b6447c_0 ...
using -f (force) option
installing: gmp-6.1.2-h6c8ec71_1 ...
using -f (force) option
installing: graphite2-1.3.13-h23475e2_0 ...
using -f (force) option
installing: icu-58.2-h9c2bf20_1 ...
using -f (force) option
installing: jbig-2.1-hdba287a_0 ...
using -f 

Check whether the system dependencies are installed correctly:

In [59]:
# some sanity checks
!conda --version
!protoc --version
!gcc-4.9 --version

conda 4.8.3
libprotoc 3.5.1
gcc-4.9 (GCC) 4.9.1
Copyright (C) 2014 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Clone the DensePose project and build it:

In [60]:
%%time
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/facebookresearch/DensePose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone project
  !git clone -q --depth 1 $git_repo_url
  # install dependencies
  !cd $project_name && pip install -q -r requirements.txt
  # update CMakeLists.txt
  cmakelists_txt_content = """
cmake_minimum_required(VERSION 2.8.12 FATAL_ERROR)
set(Caffe2_DIR "/content/anaconda2/lib/python2.7/site-packages/torch/share/cmake/Caffe2/")
find_package(Caffe2 REQUIRED)

include_directories("/content/anaconda2/lib/python2.7/site-packages/torch/lib/include")
include_directories("/content/anaconda2/include")
include_directories("/content/pytorch")

add_library(libprotobuf STATIC IMPORTED)
set(PROTOBUF_LIB "/content/anaconda2/lib/libprotobuf.a")
set_property(TARGET libprotobuf PROPERTY IMPORTED_LOCATION "${PROTOBUF_LIB}")

if (${CAFFE2_VERSION} VERSION_LESS 0.8.2)
  # Pre-0.8.2 caffe2 does not have proper interface libraries set up, so we
  # will rely on the old path.
  message(WARNING
      "You are using an older version of Caffe2 (version " ${CAFFE2_VERSION}
      "). Please consider moving to a newer version.")
  include(cmake/legacy/legacymake.cmake)
  return()
endif()

# Add compiler flags.
set(CMAKE_C_FLAGS "${CMAKE_C_FLAGS} -std=c11")
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -std=c++11 -O2 -fPIC -Wno-narrowing")

# Print configuration summary.
include(cmake/Summary.cmake)
detectron_print_config_summary()

# Collect custom ops sources.
file(GLOB CUSTOM_OPS_CPU_SRCS ${CMAKE_CURRENT_SOURCE_DIR}/detectron/ops/*.cc)
file(GLOB CUSTOM_OPS_GPU_SRCS ${CMAKE_CURRENT_SOURCE_DIR}/detectron/ops/*.cu)

# Install custom CPU ops lib.
add_library(
     caffe2_detectron_custom_ops SHARED
     ${CUSTOM_OPS_CPU_SRCS})

target_link_libraries(caffe2_detectron_custom_ops caffe2_library libprotobuf)
install(TARGETS caffe2_detectron_custom_ops DESTINATION lib)

# Install custom GPU ops lib, if gpu is present.
if (CAFFE2_USE_CUDA OR CAFFE2_FOUND_CUDA)
  # Additional -I prefix is required for CMake versions before commit (< 3.7):
  # https://github.com/Kitware/CMake/commit/7ded655f7ba82ea72a82d0555449f2df5ef38594
  list(APPEND CUDA_INCLUDE_DIRS -I${CAFFE2_INCLUDE_DIRS})
  CUDA_ADD_LIBRARY(
      caffe2_detectron_custom_ops_gpu SHARED
      ${CUSTOM_OPS_CPU_SRCS}
      ${CUSTOM_OPS_GPU_SRCS})

  target_link_libraries(caffe2_detectron_custom_ops_gpu caffe2_gpu_library libprotobuf)
  install(TARGETS caffe2_detectron_custom_ops_gpu DESTINATION lib)
endif()"""
  open(join(project_name, 'CMakeLists.txt'), 'w').write(cmakelists_txt_content)
  # build
  !cd $project_name && make
  !cd $project_name && make ops
  # download dense pose data
  !cd $project_name/DensePoseData && bash get_densepose_uv.sh

    100% |████████████████████████████████| 28.2MB 1.1MB/s 
python2 setup.py develop --user
Compiling detectron/utils/cython_bbox.pyx because it changed.
Compiling detectron/utils/cython_nms.pyx because it changed.
[1/2] Cythonizing detectron/utils/cython_bbox.pyx
/content/anaconda2/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/DensePose/detectron/utils/cython_bbox.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing detectron/utils/cython_nms.pyx
/content/anaconda2/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/DensePose/detectron/utils/cython_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating Detectron.egg-info
writing Detectron.eg

Check whether the DensePose installation was successful:

In [61]:
!python2 $project_name/detectron/tests/test_spatial_narrow_as_op.py
!python2 $project_name/detectron/tests/test_zero_even_op.py

No handlers could be found for logger "caffe2.python.net_drawer"
net_drawer will not run correctly. Please install the correct dependencies.
[E init_intrinsics_check.cc:43] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature avx2 is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature fma is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
Found Detectron ops lib: /content/anaconda2/lib/python2.7/site-packages/lib/libcaffe2_detectron_ops_gpu.so
...
----------------------------------------------------------------------
Ran 3 tests in 4.107s

OK
[E init_intrinsics_check.cc:43] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it.

In [96]:
# From https://github.com/trrahul/densepose-video/blob/master/infer_vid.py
!wget https://raw.githubusercontent.com/trrahul/densepose-video/master/infer_vid.py -O /content/DensePose/tools/infer_vid.py

--2020-03-29 17:21:45--  https://raw.githubusercontent.com/trrahul/densepose-video/master/infer_vid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4909 (4.8K) [text/plain]
Saving to: ‘/content/DensePose/tools/infer_vid.py’

/content/DensePose/ 100%[===================>]   4.79K  --.-KB/s    in 0s      

2020-03-29 17:21:46 (64.2 MB/s) - ‘/content/DensePose/tools/infer_vid.py’ saved [4909/4909]



In [0]:
import os
from os.path import exists, join, basename, splitext
import ipywidgets as widgets

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
DATASET_PATH = '/content/drive/My Drive/DATASET_DEP/'
DATASET_SYNC_PATH = DATASET_PATH+'SYNC/'

In [65]:
groups_list = os.listdir(DATASET_SYNC_PATH)

GROUP_ID = widgets.Dropdown(
    options=groups_list,
    value=groups_list[-1],
    description='Group:',
    disabled=False,
)
display(GROUP_ID)

Dropdown(description=u'Group:', index=1, options=('3CLC9VWR', '3CLC9VWR_SAMPLE'), value='3CLC9VWR_SAMPLE')

In [0]:
OPENPOSE_PATH =  DATASET_PATH + 'OPENPOSE/' + GROUP_ID.value + '/'
DENSEPOSE_PATH = DATASET_PATH + 'DENSEPOSE/' + GROUP_ID.value + '/'
GROUP_PATH = DATASET_SYNC_PATH + GROUP_ID.value + "/"

TASKS_DIRS_PATHS = dict()
for task_dir in [path for path in [GROUP_PATH+_dir for _dir in os.listdir(GROUP_PATH)]]:
  task = task_dir.split('/')[-1]
  TASKS_DIRS_PATHS[task] = task_dir+'/'

TASK_FRAMES_PATH = dict()
for task in TASKS_DIRS_PATHS:
  TASK_FRAMES_PATH[task] = [TASKS_DIRS_PATHS[task]+frames_dir for frames_dir in os.listdir(TASKS_DIRS_PATHS[task]) if 'frames_' in frames_dir]
  TASK_FRAMES_PATH[task].sort()

TASK_VIDEOS_PATH = dict()
for task in TASKS_DIRS_PATHS:
  TASK_VIDEOS_PATH[task] = [TASKS_DIRS_PATHS[task]+videos_path for videos_path in os.listdir(TASKS_DIRS_PATHS[task]) if '.avi' in videos_path]
  TASK_VIDEOS_PATH[task].sort()

In [0]:
# Split videos into frames
# if not all([os.path.exists(_dir) for _dir in TASKS_DIRS_PATHS]):
#   for task_path in TASKS_DIRS_PATHS:
#     tasks_videos = [vid_file for vid_file in os.listdir(task_path) if '.avi' in vid_file]
#     tasks_videos.sort()
#     for idx, task_video in enumerate(tasks_videos):
#       video_file = task_path + task_video
#       output_path = task_path + 'frames_pc'+str(idx+1) + '/'
#       try:
#         os.makedirs(output_path, exist_ok=True)
#       except:
#         raise
#       print(video_file, output_path)
#       !ffmpeg -i {video_file} -f image2 {output_path}image-%07d.png

In [93]:
%%time
for task in TASK_FRAMES_PATH:
  for frames_path_idx, frames_path in enumerate(TASK_FRAMES_PATH[task]):
    input_frames_path = frames_path+'/'
    output_path = DENSEPOSE_PATH + task_name + '/' + 'pc'+str(frames_path_idx+1)
    print(input_frames_path, output_path)
    try:
      os.makedirs(output_path)
    except:
      pass
    !cd $project_name && python2 tools/infer_simple.py \
      --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
      --output-dir "{output_path}" \
      --image-ext png \
      --wts https://dl.fbaipublicfiles.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
      "{input_frames_path}"

    # !python2 tools/infer_vid.py     
    # --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml     
    # --output-dir DensePoseData/infer_out/    
    # --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl 
    # --input-file filename

Streaming output truncated to the last 5000 lines.
INFO infer_simple.py: 113:  | misc_bbox: 0.000s
IUV written to:  /content/drive/My Drive/DATASET_DEP/DENSEPOSE/3CLC9VWR_SAMPLE/task_1/pc2/frame_0000149_IUV.png
INFO infer_simple.py: 103: Processing /content/drive/My Drive/DATASET_DEP/SYNC/3CLC9VWR_SAMPLE/task_2/frames_pc2/frame_0000241.png -> /content/drive/My Drive/DATASET_DEP/DENSEPOSE/3CLC9VWR_SAMPLE/task_1/pc2/frame_0000241.png.pdf
INFO infer_simple.py: 111: Inference time: 0.315s
INFO infer_simple.py: 113:  | im_detect_bbox: 0.173s
INFO infer_simple.py: 113:  | im_detect_body_uv: 0.141s
INFO infer_simple.py: 113:  | misc_bbox: 0.000s
IUV written to:  /content/drive/My Drive/DATASET_DEP/DENSEPOSE/3CLC9VWR_SAMPLE/task_1/pc2/frame_0000241_IUV.png
INFO infer_simple.py: 103: Processing /content/drive/My Drive/DATASET_DEP/SYNC/3CLC9VWR_SAMPLE/task_2/frames_pc2/frame_0000056.png -> /content/drive/My Drive/DATASET_DEP/DENSEPOSE/3CLC9VWR_SAMPLE/task_1/pc2/frame_0000056.png.pdf
INFO infer_s

In [0]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np

# im  = cv2.imread(join(project_name, 'DensePoseData/demo_data/demo_im.jpg'))
# IUV = cv2.imread(join(project_name, 'DensePoseData/infer_out/demo_im_IUV.png'))
# INDS = cv2.imread(join(project_name, 'DensePoseData/infer_out/demo_im_INDS.png'),  0)

# fig = plt.figure(figsize=[20, 20])
# plt.imshow( im[:,:,::-1] )
# plt.contour( IUV[:,:,1]/256.,10, linewidths = 1 )
# plt.contour( IUV[:,:,2]/256.,10, linewidths = 1 )
# plt.axis('off') ; plt.show()